# Install dependencies
pip install langchain
pip install tiktoken
pip install transformers

In [1]:
pdfUrl = 'gpt-4.pdf'

In [2]:
""" Read the pdf file """

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdfUrl)
data = []
all_content = []
pages = loader.load_and_split()
if not len(pages):
    raise ValueError("No data found")
for page in pages:
    content = page.page_content
    # content = clean_string(content)
    meta_data = page.metadata
    meta_data["url"] = pdfUrl
    data.append(
        {
            "content": content,
            "meta_data": meta_data,
        }
    )
    all_content.append(content)
# doc_id = hashlib.sha256((" ".join(all_content) + url).encode()).hexdigest()

print(data)

"""
data = [
  {
    content: string;
    meta_data: {
      source: string;
      page: number;
      url: string;
    }
  }
]
"""


[{'content': 'GPT-4 Technical Report\nOpenAI∗\nAbstract\nWe report the development of GPT-4, a large-scale, multimodal model which can\naccept image and text inputs and produce text outputs. While less capable than\nhumans in many real-world scenarios, GPT-4 exhibits human-level performance\non various professional and academic benchmarks, including passing a simulated\nbar exam with a score around the top 10% of test takers. GPT-4 is a Transformer-\nbased model pre-trained to predict the next token in a document. The post-training\nalignment process results in improved performance on measures of factuality and\nadherence to desired behavior. A core component of this project was developing\ninfrastructure and optimization methods that behave predictably across a wide\nrange of scales. This allowed us to accurately predict some aspects of GPT-4’s\nperformance based on models trained with no more than 1/1,000th the compute of\nGPT-4.\n1 Introduction\nThis technical report presents GPT-4,

In [10]:
""" Tokenize the data """

from transformers import AutoTokenizer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

text_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=0, 
    model_name="sentence-transformers/all-MiniLM-L6-v2", 
    tokens_per_chunk=256
)

chunked_data = []

for page in data:
  chunks = text_splitter.split_text(page["content"])
  for chunk in chunks:
    chunked_data.append({
      "content": chunk,
      "meta_data": page["meta_data"]
    })

print(chunked_data)

[{'content': 'gpt - 4 technical report openai∗ abstract we report the development of gpt - 4, a large - scale, multimodal model which can accept image and text inputs and produce text outputs. while less capable than humans in many real - world scenarios, gpt - 4 exhibits human - level performance on various professional and academic benchmarks, including passing a simulated bar exam with a score around the top 10 % of test takers. gpt - 4 is a transformer - based model pre - trained to predict the next token in a document. the post - training alignment process results in improved performance on measures of factuality and adherence to desired behavior. a core component of this project was developing infrastructure and optimization methods that behave predictably across a wide range of scales. this allowed us to accurately predict some aspects of gpt - 4 ’ s performance based on models trained with no more than 1 / 1, 000th the compute of gpt - 4. 1 introduction this technical report pr

In [21]:
""" Embed all chunks """

import pyembeddings
pyembeddings.init('f257bbe3-bbc3-4885-90a1-3bd48e6ec591')

gen = pyembeddings.Generator()
gen.set_model("MiniLM")

# chunked_data = chunked_data[:10]

embeddings = []

for chunk in chunked_data:
  embeddings.append(gen.embed(chunk["content"])[0])

print(len(embeddings))
print(len(chunked_data))

print(embeddings[0])

db = pyembeddings.Database()
collection = db.create_collection('pdf_collection', 'MiniLM')

documents = []
metadatas = []
ids = []

for index, chunk in enumerate(chunked_data):
  documents.append(chunk["content"])
  metadatas.append(chunk["meta_data"])
  ids.append(str(index))

print('embeddings', embeddings)

addAll = collection.add(
  documents=documents,
  embeddings=embeddings,
  metadatas=metadatas,
  ids=ids,
)

print(addAll)


10
10
[-0.057403162121772766, -0.04463536664843559, -0.006716681178659201, -0.007699054665863514, 0.044755760580301285, -0.05124007537961006, -0.016915036365389824, 0.07832682132720947, 0.05007052794098854, -0.04052967578172684, -0.05890699848532677, -0.020623521879315376, 0.01638336293399334, 0.10389180481433868, -0.016955144703388214, -0.06533598154783249, 0.004229317419230938, 0.020080504938960075, -0.1000128760933876, -0.05718523636460304, -0.02374526485800743, 0.04605412110686302, 0.06637968122959137, -0.023126501590013504, 0.02326352894306183, 0.04151042178273201, -0.00249940506182611, -0.10326822102069855, -0.014342015609145164, -0.04879685118794441, 0.019547652453184128, -0.03638960048556328, 0.04739947244524956, 0.09669756144285202, -0.11693989485502243, 0.08863189816474915, -0.0512777678668499, -0.028218254446983337, 0.01511848159134388, -0.06708843261003494, -0.005175513215363026, -0.031087461858987808, -0.007732637692242861, 0.014337843284010887, 0.13786906003952026, -0.020

In [20]:
db.delete_collection('pdf_collection')

In [26]:
query_string = "What is the performance of GPT-4"
query_embedding = gen.embed(query_string)[0]

query = collection.query(
  embedding=query_embedding, 
  n_results=5
)

print(query)

{'ids': [['7', '8', '1', '4', '2']], 'distances': [[0.42010308983124245, 0.44530814018646614, 0.4864186073965727, 0.49790227140149945, 0.5074567795498237]], 'embeddings': None, 'metadatas': [[{'page': 2, 'source': 'gpt-4.pdf', 'url': 'gpt-4.pdf'}, {'page': 2, 'source': 'gpt-4.pdf', 'url': 'gpt-4.pdf'}, {'page': 0, 'source': 'gpt-4.pdf', 'url': 'gpt-4.pdf'}, {'page': 1, 'source': 'gpt-4.pdf', 'url': 'gpt-4.pdf'}, {'page': 0, 'source': 'gpt-4.pdf', 'url': 'gpt-4.pdf'}]], 'documents': [['observed prediction gpt - 4 100p 10n 1µ 100µ 0. 01 1 compute1. 02. 03. 04. 05. 06. 0bits per wordopenai codebase next word predictionfigure 1. performance of gpt - 4 and smaller models. the metric is ﬁnal loss on a dataset derived from our internal codebase. this is a convenient, large dataset of code tokens which is not contained in the training set. we chose to look at loss because it tends to be less noisy than other measures across different amounts of training compute. a power law ﬁt to the smaller m